In [1]:
from database_cloudant import CloudantDB
from cloudant.design_document import DesignDocument
db10 = CloudantDB("topwordsmonth3")
#db10.add_record(cdict,key="2021-04")


print(db10.curDB["2021-04"])
#print()
#print(db10.get_data())

Username: admin
Databases: ['topwordsmonth3', 'topwordstotal', 'tweets_covid', 'tweets_keywords', 'tweets_no_keywords', 'tweets_raw', 'tweets_vaccine']
Accessing db: topwordsmonth3
{'_id': '2021-04', '_rev': '1-cf0b2bd10358c18c2d8196bdbe77d4d8', 'Adelaide': {'Adelaide': 281, 'Australia': 203, 'South': 163, 'posted': 141, 'photo': 134, 'today': 125, 'people': 82, 'great': 80, 'Happy': 68, 'video': 68}, 'Ballarat': {'#walkies': 24, 'Ballarat': 23, 'Abstract': 16, '#zerowaste': 14, '#recycle': 13, '#MAFS': 12, '#abstractart': 12, '#artist': 11, '#recycling': 11, '#artwork': 11}, 'Brisbane': {'posted': 961, 'photo': 938, 'Brisbane': 489, 'Queensland': 476, 'Australia': 350, 'today': 201, 'people': 126, 'great': 119, 'night': 114, 'first': 111}, 'Bunbury': {'Bunbury': 7, 'Western': 4, 'Australia': 4, 'posted': 3, 'photo': 3, '#housesitting': 2, 'walking': 2, 'today': 2, 'Taking': 1, 'break': 1}, 'Canberra': {'Canberra': 168, 'posted': 101, 'photo': 97, 'Australian': 71, 'Australia': 65, 'gr

In [ ]:
with open('top10wordsfreq.json', 'w') as f:
    json.dump(popdict, f)

    
f.close()